In [1]:
import optuna
import plotly
import numpy as np
import matplotlib.pyplot as plt
import os
from keras.utils.vis_utils import plot_model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, regularizers, initializers, callbacks
import random as python_random
from numpy.random import seed
np.random.seed(123)
tf.random.set_seed(11)
python_random.seed(123)
os.environ['PYTHONHASHSEED'] = '0'
from keras import backend as K
from keras.backend import set_session
from keras.backend import clear_session
from keras.backend import get_session
import tensorflow
import gc
from sklearn.metrics import confusion_matrix
import itertools
from keras.callbacks import ModelCheckpoint
import json
from sklearn.model_selection import KFold

In [2]:
def codificacion(tamano,unico=True,numero=0):
    #Vector de unos
    if unico==True:
        Y_train=np.ones(tamano)*numero
    else:
        for i in range(0,5):
            if i==0:
                Y_train=np.ones(tamano)*i
            else:
                vectorUnos=np.ones(tamano)
                Y_train=np.insert(Y_train,len(Y_train),vectorUnos*i,axis=0)
    return Y_train
def load_data_test_gru():
    # load your data using this function
    path='C:\\Users\jdani\\Documents\\TrabajoDeGrado\\BDConEspectrograma\\MatricesNumpyGuardadas_NA\\ECG_'
    #importando patologias
    ECG_Normal_Test=np.load(path+'Normal\\ECG_Normal_test.npy')
    ECG_SBR_Test=np.load(path+'SBR\\ECG_SBR_Original.npy')
    ECG_SBR_Test=ECG_SBR_Test[300:360,:,:]
    ECG_AFIB_Test=np.load(path+'AFIB\\ECG_AFIB_test.npy')
    ECG_STD_Test=np.load(path+'STD\\ECG_STD_test.npy')
    ECG_STE_Test=np.load(path+'STE\\ECG_STE_test.npy')

    y_Normal_Test=codificacion(len(ECG_Normal_Test),True,0)
    y_SBR_Test=codificacion(len(ECG_SBR_Test),True,1)
    y_AFIB_Test=codificacion(len(ECG_AFIB_Test),True,2)
    y_STD_Test=codificacion(len(ECG_STD_Test),True,3)
    y_STE_Test=codificacion(len(ECG_STE_Test),True,4)
    
    #Uniendo canales
    ECG_Normal_Test=np.concatenate((ECG_Normal_Test[:,:,:,0],ECG_Normal_Test[:,:,:,1]),axis=1)
    ECG_SBR_Test=np.concatenate((ECG_SBR_Test[:,:,:,0],ECG_SBR_Test[:,:,:,1]),axis=1)
    ECG_AFIB_Test=np.concatenate((ECG_AFIB_Test[:,:,:,0],ECG_AFIB_Test[:,:,:,1]),axis=1)
    ECG_STE_Test=np.concatenate((ECG_STE_Test[:,:,:,0],ECG_STE_Test[:,:,:,1]),axis=1)
    ECG_STD_Test=np.concatenate((ECG_STD_Test[:,:,:,0],ECG_STD_Test[:,:,:,1]),axis=1)

    #Generando X_train
    X_test=np.concatenate((ECG_Normal_Test,
                                ECG_SBR_Test,
                                ECG_AFIB_Test,
                                ECG_STE_Test,
                                ECG_STD_Test))

    Y_test=np.insert(y_Normal_Test,len(y_Normal_Test),y_SBR_Test,axis=0)
    Y_test=np.insert(Y_test,len(Y_test),y_AFIB_Test,axis=0)
    Y_test=np.insert(Y_test,len(Y_test),y_STD_Test,axis=0)
    Y_test=np.insert(Y_test,len(Y_test),y_STE_Test,axis=0)    

    print('Tamaño datos test: '+ str(X_test.shape))
    print('Tamaño etiquetas de test: '+str(Y_test.shape))
    
    X_test=np.moveaxis(X_test,1,2)   
    return X_test,Y_test
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [3]:
X_val, y_val = load_data_test_gru()
y_val= keras.utils.to_categorical(y_val)
print(X_val.shape)
print(y_val.shape)

Tamaño datos test: (6013, 102, 241)
Tamaño etiquetas de test: (6013,)
(6013, 241, 102)
(6013, 5)


# Arquitectura convolucional sin normalización

In [4]:
from sklearn.model_selection import KFold
import numpy as np
from keras.models import load_model

# Load the previously trained Keras model
model_file = 'C:\\Users\\jdani\\Documents\\TrabajoDeGrado\\Arquitecturas\\ArquitecturaRecurrente\\ModelosCheckpointsOptuna\\NA\\mejor_modelo_trial'
modelo='8'
model = keras.models.load_model(model_file+modelo+'.h5',custom_objects={'f1': f1})

# Load the data for cross-validation
X, y = load_data_test_gru()
y= keras.utils.to_categorical(y)
# Define the number of folds for cross-validation
k = 5

# Initialize the KFold class
kf = KFold(n_splits=k)

perdida=[]
precision=[]
auc=[]
f1_score=[]

# Iterate over the folds
for train_indices, val_indices in kf.split(X):
    # Split the data into training and validation sets
    X_train, y_train = X[train_indices], y[train_indices]
    X_val, y_val = X[val_indices], y[val_indices]

    # Evaluate the model on the validation set
    metricas = model.evaluate(X_val, y_val, verbose=0)
    perdida.append(metricas[0])
    precision.append(metricas[1]*100)
    auc.append(metricas[2]*100)
    f1_score.append(metricas[3]*100)
    
    print(metricas)
print()
print('Validation loss: {:.2f} +- {:.2f}'.format(np.mean(perdida), np.std(perdida)))
print('Validation accuracy: {:.2f}% +- {:.2f}%'.format(np.mean(precision), np.std(precision)))
print('Validation AUC: {:.2f}% +- {:.2f}%'.format(np.mean(auc), np.std(auc)))
print('Validation f1_score: {:.2f}% +- {:.2f}%'.format(np.mean(f1_score), np.std(f1_score)))

Tamaño datos test: (6013, 102, 241)
Tamaño etiquetas de test: (6013,)
[0.28063511848449707, 0.9983375072479248, 0.9999468326568604, 0.9983552694320679]
[0.654721200466156, 0.8877805471420288, 0.9718964695930481, 0.7989470362663269]
[0.29116445779800415, 0.9817124009132385, 0.9994387030601501, 0.9800031781196594]
[8.10855484008789, 0.4633943438529968, 0.5973940491676331, 0.4580592215061188]
[5.654313087463379, 0.43427619338035583, 0.7056654095649719, 0.4296628534793854]

Validation loss: 3.00 +- 3.27
Validation accuracy: 75.31% +- 25.14%
Validation AUC: 85.49% +- 16.98%
Validation f1_score: 73.30% +- 24.63%
